In [ ]:
import pickle
import pprint
import os
from src.simulate import experiment
import src.params as params
from src.plot_visuals import generate_gif, generate_pdf
from src.plot_metrics import (
    plot_metrics,
    plot_bar,
    plot_results_matrix,
    plot_multi_corrs,
    ttests,
)

# make sure pyplot uses retina display
%config InlineBackend.figure_format = 'retina'

### Define hyperparameters and experiment conditions

In [ ]:
pprint.pprint(params.SimHyperparams())
pprint.pprint(params.PlotHyperparams())

In [ ]:
inhibit_range = [0.0, 0.03, 0.06, 0.13, 0.25, 0.5]
excite_range = [0.0, 0.03, 0.06, 0.13, 0.25, 0.5]
dose_names = ["zero", "min", "light", "medium", "heavy", "max"]
compare_ranges = [[0, 0], [0, 3], [3, 0], [0, 5], [5, 0], [3, 3], [5, 5], [5, 4]]

plastic_strengths = [0.1, 0.5, 1.0]
homeo_strengths = [0.01, 0.05, 0.1]
num_grad_steps = [1, 10, 100]
landscapes = ["noise", "sine", "tangent", "log", "twisted_sine", "swirl"]

# plasticity variable strength sweep.
plastic_sweep = {}
for ps in plastic_strengths:
    plastic_sweep[f"plastic_{ps}"] = {
        f"1a_{dose_names[one_a]}_2a_{dose_names[two_a]}": params.SimHyperparams(
            inhibit_str=inhibit_range[one_a],
            excite_str=excite_range[two_a],
            plastic_str=ps,
            verbose=True if (one_a == 0 and two_a == 0) else False,
        )
        for one_a, two_a in compare_ranges
    }
# homeostatic variable strength sweep.
homeo_sweep = {}
for hs in homeo_strengths:
    homeo_sweep[f"homeo_{hs}"] = {
        f"1a_{dose_names[one_a]}_2a_{dose_names[two_a]}": params.SimHyperparams(
            inhibit_str=inhibit_range[one_a],
            excite_str=excite_range[two_a],
            constraint_str=hs,
            verbose=True if (one_a == 0 and two_a == 0) else False,
        )
        for one_a, two_a in compare_ranges
    }
# number of gradient steps sweep.
grad_step_sweep = {}
for gs in num_grad_steps:
    grad_step_sweep[f"grad_step_{gs}"] = {
        f"1a_{dose_names[one_a]}_2a_{dose_names[two_a]}": params.SimHyperparams(
            inhibit_str=inhibit_range[one_a],
            excite_str=excite_range[two_a],
            grad_step_num=gs,
            verbose=True if (one_a == 0 and two_a == 0) else False,
        )
        for one_a, two_a in compare_ranges
    }
# energy function surface pattern (landscape) sweep.
landscape_sweep = {}
for ls in landscapes:
    landscape_sweep[f"landscape_{ls}"] = {
        f"1a_{dose_names[one_a]}_2a_{dose_names[two_a]}": params.SimHyperparams(
            inhibit_str=inhibit_range[one_a],
            excite_str=excite_range[two_a],
            surface_pattern=ls,
            verbose=True if (one_a == 0 and two_a == 0) else False,
        )
        for one_a, two_a in compare_ranges
    }

experiment_choices = {
    "plastic_sweep": plastic_sweep,
    "homeo_sweep": homeo_sweep,
    "grad_step_sweep": grad_step_sweep,
    "landscape_sweep": landscape_sweep,
}

plot_name = "plastic_sweep"
experiment_dict = experiment_choices[plot_name]
plot_params = params.PlotHyperparams()
print(f"Experiments to run: \n{list(experiment_dict.keys())})")

### Run experiment and save results

In [ ]:
# run experiment and save results
for meta_exp_name, exps in experiment_dict.items():
    print(f"Running experiment with {meta_exp_name}.")
    metric_results = {}
    sim_results = {}
    all_results = {}
    for exp_name, params in exps.items():
        (
            all_metrics,
            avg_metrics,
            std_error_metrics,
            e_mods,
            x,
            y,
            zs,
            e_star,
        ) = experiment(params)
        all_results[exp_name] = all_metrics
        metric_results[exp_name] = (avg_metrics, std_error_metrics)
        sim_results[exp_name] = (e_mods, x, y, zs, e_star)
        print("Finished experiment: ", exp_name)

    if not os.path.exists("./output/results"):
        os.makedirs("./output/results")

    with open(f"./output/results/metric_results_{meta_exp_name}.pkl", "wb") as f:
        pickle.dump(metric_results, f)

    with open(f"./output/results/all_results_{meta_exp_name}.pkl", "wb") as f:
        pickle.dump(all_results, f)

    if plot_params.generate:
        with open(f"./output/results/sim_results_{meta_exp_name}.pkl", "wb") as f:
            pickle.dump(sim_results, f)

### Perform statistical analysis

In [ ]:
def get_experiment(exp_name, metric, summary_type, drug_steps, all_results):
    if summary_type == "final":
        return all_results[exp_name][metric][:, -1]
    else:
        return all_results[exp_name][metric][:, :drug_steps].mean(axis=1)


for meta_exp_name, exps in experiment_dict.items():
    print(f"Calculating statistics for {meta_exp_name}")

    # load all_results
    with open(f"./output/results/all_results_{meta_exp_name}.pkl", "rb") as f:
        all_results = pickle.load(f)

    # run t-tests
    metric = "div_monotonicity"
    summary_type = "final"  # final, avg, max, min
    ttests(all_results, exps, metric, summary_type)

    # run correlation plots
    x_metrics = ["energy", "gradient_mags", "local_minima", "state_counts"]
    y_metric = "divergence"
    summary_type_xs = ["avg", "avg", "avg", "final"]  # final, avg, max, min
    summary_type_y = "final"  # final, avg, max, min
    plot_multi_corrs(
        all_results,
        exps,
        x_metrics,
        y_metric,
        summary_type_xs,
        summary_type_y,
        meta_exp_name,
    )

### Plot results and save figures

In [ ]:
for meta_exp_name, exps in experiment_dict.items():
    # load metric results from pickle file
    with open(f"./output/results/metric_results_{meta_exp_name}.pkl", "rb") as f:
        metric_results = pickle.load(f)

    if len(exps) == 49 or len(exps) == 36:
        plot_results_matrix(metric_results, inhibit_range, excite_range)
    else:
        plot_metrics(
            metric_results,
            plt_name=meta_exp_name,
            minimal_timeseries=plot_params.minimal_timeseries,
            format="pdf",
            render_titles=False,
            exp_dict=exps,
        )
        # Plot select bar charts
        keys = ["divergence", "div_monotonicity"]
        titles = ["Final Divergence Value", "Cumulative Divergence Increase"]
        plot_bar(metric_results, keys, titles, plt_name=meta_exp_name, format="pdf")

### Generate visualization images

In [ ]:
for meta_exp_name in experiment_dict.keys():
    if plot_params.generate:
        # load sim results from pickle file
        with open(f"./output/results/sim_results_{meta_exp_name}.pkl", "rb") as f:
            sim_results = pickle.load(f)

        for exp_name, results in sim_results.items():
            e_mods, x, y, zs, e_star = results
            if plot_params.output_type == "pdf":
                generate_pdf(e_mods, e_star, x, y, zs, plot_params, exp_name)
            else:
                generate_gif(e_mods, x, y, zs, e_star, plot_params, exp_name)
            print(f"Generated {exp_name} plots")